In [1]:
from hybrid import *
from Data_Handler.DataReader import DataReader
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from tqdm import tqdm
from evaluator import evaluate
# from Evaluation import Evaluator
from HyperparameterTuning.run_hyperparameter_search import runHyperparameterSearch_Collaborative
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender


from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.DataIO import DataIO
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from skopt.space import Real, Integer, Categorical
import os

In [2]:
# Read & split data
dataReader = DataReader()

target = dataReader.load_target()

URM = dataReader.load_augmented_binary_urm()
URM_aug,icm = dataReader.pad_with_zeros_ICMandURM(URM)

>>> number of target users: 41116
4877


In [3]:
URM_train_aug, URM_validation = split_train_in_two_percentage_global_sample(URM_aug, train_percentage = 0.9)
URM_train_pow = dataReader.stackMatrixes(URM_train_aug)


evaluator_validation = EvaluatorHoldout(URM_validation, [10])

EvaluatorHoldout: Ignoring 2782 ( 6.7%) Users that have less than 1 test interactions


In [4]:
UserKNNCF = UserKNNCFRecommender(URM_train_aug)
UserKNNCF.fit()

RP3beta_pow = RP3betaRecommender(URM_train_pow)
RP3beta_pow.fit(alpha=0.3648761546066018,beta=0.5058870363874656, topK=480, normalize_similarity=True)

S_SLIM = SLIMElasticNetRecommender(URM_train_pow)
S_SLIM.fit()

EASE_R = EASE_R_Recommender(URM_train_aug)
EASE_R.fit()

UserKNNCFRecommender: URM Detected 3461 (12.4%) items with no interactions.
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 41629 (100.0%), 670.43 column/sec. Elapsed time 1.03 min
RP3betaRecommender: Similarity column 27968 (100.0%), 1724.98 column/sec. Elapsed time 16.21 sec
SLIMElasticNetRecommender: Processed 6647 (23.8%) in 5.00 min. Items per second: 22.15
SLIMElasticNetRecommender: Processed 7873 (28.2%) in 13.25 min. Items per second: 9.91
SLIMElasticNetRecommender: Processed 14806 (52.9%) in 18.25 min. Items per second: 13.52
SLIMElasticNetRecommender: Processed 21884 (78.2%) in 23.25 min. Items per second: 15.69
SLIMElasticNetRecommender: Processed 27968 (100.0%) in 25.19 min. Items per second: 18.50
EASE_R_Recommender: URM Detected 3461 (12.4%) items with no interactions.
EASE_R_Recommender: Fitting model... 
EASE_R_Recommender: Fitting model... done in 4.72 min


In [5]:
recommender_class = HybridRecommender_5

output_folder_path = "result_experiments/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

n_cases = 400
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"
cutoff_to_optimize = 10

In [6]:
hyperparameters_range_dictionary = {
    #"ItemKNNCF_tier1_weight": Real(0,1),
    "UserKNNCF_tier1_weight": Real(0,1),
    "RP3beta_pow_tier1_weight": Real(0,1),
    "EASE_R_tier1_weight": Real(0,1),
    
    #"ItemKNNCF_tier2_weight": Real(0,1),
    "UserKNNCF_tier2_weight": Real(0,1),
    "RP3beta_pow_tier2_weight": Real(0,1),
    "EASE_R_tier2_weight": Real(0,1),


    "RP3beta_pow_tier3_weight": Real(0,1),
    "S_SLIM_tier3_weight": Real(0,1),
    "EASE_R_tier3_weight": Real(0,1),

    "S_SLIM_tier4_weight": Real(0,1),
    "EASE_R_tier4_weight": Real(0,1),

}


In [7]:
# create a bayesian optimizer object, we pass the recommender and the evaluator
hyperparameterSearch = SearchBayesianSkopt(recommender_class,
                                           evaluator_validation=evaluator_validation)

# provide data needed to create instance of model (one on URM_train, the other on URM_all)
recommender_input_args = SearchInputRecommenderArgs(
    # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_POSITIONAL_ARGS=[URM_train_aug,URM_train_pow, UserKNNCF,RP3beta_pow,S_SLIM,EASE_R],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={},
    EARLYSTOPPING_KEYWORD_ARGS={},
)

recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[URM_train_aug,URM_train_pow, UserKNNCF,RP3beta_pow,S_SLIM,EASE_R],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={},
    EARLYSTOPPING_KEYWORD_ARGS={},
)

In [8]:

# let's run the bayesian search
hyperparameterSearch.search(recommender_input_args,
                            recommender_input_args_last_test=recommender_input_args_last_test,
                            hyperparameter_search_space=hyperparameters_range_dictionary,
                            n_cases=n_cases,
                            n_random_starts=n_random_starts,
                            save_model="last",
                            output_folder_path=output_folder_path,  # Where to save the results
                            output_file_name_root=recommender_class.RECOMMENDER_NAME,  # How to call the files
                            metric_to_optimize=metric_to_optimize,
                            cutoff_to_optimize=cutoff_to_optimize,
                            )

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'UserKNNCF_tier1_weight': 0.5811119709237523, 'RP3beta_pow_tier1_weight': 0.9500333542745011, 'EASE_R_tier1_weight': 0.2463220022486991, 'UserKNNCF_tier2_weight': 0.9968526713665654, 'RP3beta_pow_tier2_weight': 0.7930063783462498, 'EASE_R_tier2_weight': 0.4173752373795173, 'RP3beta_pow_tier3_weight': 0.005791884238670942, 'S_SLIM_tier3_weight': 0.2837919010371129, 'EASE_R_tier3_weight': 0.9614304012274617, 'S_SLIM_tier4_weight': 0.47430281060017054, 'EASE_R_tier4_weight': 0.5198498346125332}
Hybrid_Recommender_5: URM Detected 3461 (12.4%) items with no interactions.
EvaluatorHoldout: Processed 38847 (100.0%) in 1.28 min. Users per second: 506
SearchBayesianSkopt: New best config found. Config 0: {'UserKNNCF_tier1_weight': 0.5811119709237523, 'RP3beta_pow_tier1_weight': 0.9500333542745011, 'EASE_R_tier1_weight': 0.2463220022486991, 'UserKNNCF_tier2_weight': 0.9968526713665654, 'RP3beta_po

KeyboardInterrupt: 